#10 Parallel Processing Tugas Individu
---
Nama : Iqbal Biondy
NIM  : 205150601111009

> Indented block




## Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import joblib

from multiprocessing import Pool
from multiprocessing import cpu_count
from functools import partial

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

## Download dataset

source dataset : https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

setelah dataset di-download, dataset akan dimasukkan kedalam dataframe pandas

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rafiag/DTI2020/main/data/bank.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


## Preprocessing
preprocessing yang dilakukan adalah membuang fitur yang tidak diperlukan yaitu fitur duration, kemudian memastikan tidak ada null value dalam dataset, dan mengubah nilai data kategorikal menjadi float

In [ ]:
#check class distribution
df['deposit'].value_counts()

no     5873
yes    5289
Name: deposit, dtype: int64

In [ ]:
#drop 'duration' column
df = df.drop('duration', axis=1)

In [ ]:
#check missing value
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [ ]:
#scale numeric data
from sklearn.preprocessing import StandardScaler


df_bank = df.copy()

scaler = StandardScaler()
cols = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']
df_bank[cols] = scaler.fit_transform(df_bank[cols])

df_bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,deposit
0,1.491505,admin.,married,secondary,no,0.252525,yes,no,unknown,-1.265746,may,-0.554168,-0.481184,-0.36326,unknown,yes
1,1.239676,admin.,married,secondary,no,-0.459974,no,no,unknown,-1.265746,may,-0.554168,-0.481184,-0.36326,unknown,yes
2,-0.019470,technician,married,secondary,no,-0.080160,yes,no,unknown,-1.265746,may,-0.554168,-0.481184,-0.36326,unknown,yes
3,1.155733,services,married,secondary,no,0.293762,yes,no,unknown,-1.265746,may,-0.554168,-0.481184,-0.36326,unknown,yes
4,1.071790,admin.,married,tertiary,no,-0.416876,no,no,unknown,-1.265746,may,-0.186785,-0.481184,-0.36326,unknown,yes


In [ ]:
#encode categorical value with OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Encode Categorical Data
df_encoded = pd.DataFrame(encoder.fit_transform(df_bank[cat_cols]))
df_encoded.columns = encoder.get_feature_names(cat_cols)

# Replace Categotical Data with Encoded Data
df_bank = df_bank.drop(cat_cols ,axis=1)
df_bank = pd.concat([df_encoded, df_bank], axis=1)

# Encode target value
df_bank['deposit'] = df_bank['deposit'].apply(lambda x: 1 if x == 'yes' else 0)

print('Shape of dataframe:', df_bank.shape)
df_bank.head()

Shape of dataframe: (11162, 51)


,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,poutcome_other,poutcome_success,poutcome_unknown,age,balance,day,campaign,pdays,previous,deposit
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.491505,0.252525,-1.265746,-0.554168,-0.481184,-0.36326,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.239676,-0.459974,-1.265746,-0.554168,-0.481184,-0.36326,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,-0.019470,-0.080160,-1.265746,-0.554168,-0.481184,-0.36326,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.155733,0.293762,-1.265746,-0.554168,-0.481184,-0.36326,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.071790,-0.416876,-1.265746,-0.186785,-0.481184,-0.36326,1


## Model Training
Dataset akan dipecah menjadi data training dan data test menggunakan train_test_split dari sklearn.
Model yang digunakan adalah model random forest classsifier yang kemudian di-train dengan data latih. Setelah itu akan dilakukan klasifikasi dan prediksi pada sampel


In [ ]:
#set training and testing data
# Select Features
feature = df_bank.drop('deposit', axis=1)

# Select Target
target = df_bank['deposit']

# Set Training and Testing Data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature , target, 
                                                    shuffle = True, 
                                                    test_size=0.2, 
                                                    random_state=1)

print('Shape of training feature:', X_train.shape)
print('Shape of testing feature:', X_test.shape)
print('Shape of training label:', y_train.shape)
print('Shape of training label:', y_test.shape)

Shape of training feature: (8929, 50)
Shape of testing feature: (2233, 50)
Shape of training label: (8929,)
Shape of training label: (2233,)


In [ ]:
%%time
print('training model')

#instantiate model
modelML= RandomForestClassifier(n_jobs=2)


#fit the model with data
modelML.fit(X_train, y_train)

training model
CPU times: user 1.91 s, sys: 60.9 ms, total: 1.97 s
Wall time: 1.15 s


RandomForestClassifier(n_jobs=2)

In [ ]:
#perform classification and prediction on sample
predicted_model = modelML.predict(X_test)

print(classification_report(y_test,predicted_model))
report = classification_report(y_test, predicted_model, output_dict=True)

              precision    recall  f1-score   support

           0       0.70      0.80      0.75      1165
           1       0.74      0.63      0.68      1068

    accuracy                           0.72      2233
   macro avg       0.72      0.72      0.72      2233
weighted avg       0.72      0.72      0.72      2233



## Model Evaluation
Evaluasi model dilakukan dengan menggunakan cross validation score pada model yang telah dibuat

In [ ]:
%%time
n_scores = cross_val_score(
    modelML,
    X_train,
    y_train,
    scoring='accuracy',
    cv=4,
    n_jobs=1    
)

CPU times: user 1.54 s, sys: 257 ms, total: 1.8 s
Wall time: 7.01 s


In [ ]:
%%time
n_scores = cross_val_score(
    modelML,
    X_train,
    y_train,
    scoring='accuracy',
    cv=4,
    n_jobs=2    
)

CPU times: user 78.8 ms, sys: 10.7 ms, total: 89.5 ms
Wall time: 6.54 s


# Parallelize Model Prediction

In [ ]:
''''#prediction function for given dataset
def predict(data, feature_cols, clf, pred_col):
    ft = data[feature].values
    res = clf.predict(ft)
    data[pred_col] = res
    return data'''

"'#prediction function for given dataset\ndef predict(data, feature_cols, clf, pred_col):\n    ft = data[feature].values\n    res = clf.predict(ft)\n    data[pred_col] = res\n    return data"

In [ ]:
'''%%time
# normal predictions
res = predict(
    data = df_bank,
    feature_cols = feature,
    clf = modelML,
    pred_col = 'lang_prediction'
)'''

"%%time\n# normal predictions\nres = predict(\n    data = df_bank,\n    feature_cols = feature,\n    clf = modelML,\n    pred_col = 'lang_prediction'\n)"

In [ ]:
'''def parallel_prediction(fn, data, feature_cols, clf,pred_col,n_cores):
  if cpu_count < n_cores:
    raise ValueError("The number of CPU's specified exceed the amount available")
  
  df_list = np.array_split(data,n_cores)
  pool = Pool(n_cores)
  res = pool.map(partial(
      fn,
      feature_cols = feature_cols,
      clf=clf,
      pred_col = pred_col
  ), df_list)
  pool.close()
  pool.join()
  return pd.concat(res)'''

'def parallel_prediction(fn, data, feature_cols, clf,pred_col,n_cores):\n  if cpu_count < n_cores:\n    raise ValueError("The number of CPU\'s specified exceed the amount available")\n  \n  df_list = np.array_split(data,n_cores)\n  pool = Pool(n_cores)\n  res = pool.map(partial(\n      fn,\n      feature_cols = feature_cols,\n      clf=clf,\n      pred_col = pred_col\n  ), df_list)\n  pool.close()\n  pool.join()\n  return pd.concat(res)'

In [ ]:
%%time
'''res = parallel_prediction(
    fn=predict,
    data=df_bank,
    feature_cols= feature(),
    clf=modelML,
    pred_col = target,
    n_cores = 2
)'''

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 36.2 µs


'res = parallel_prediction(\n    fn=predict,\n    data=df_bank,\n    feature_cols= feature(),\n    clf=modelML,\n    pred_col = target,\n    n_cores = 2\n)'

In [ ]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,2,-1,0,unknown,yes


In [ ]:
df.to_csv('bankMarket.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  campaign   11162 non-null  int64 
 12  pdays      11162 non-null  int64 
 13  previous   11162 non-null  int64 
 14  poutcome   11162 non-null  object
 15  deposit    11162 non-null  object
dtypes: int64(6), object(10)
memory usage: 1.4+ MB


# Load data to database sqlite3


In [ ]:
import sqlite3

## Pembuatan Database dan Load data ke database
Proses pembuatan database dilakukan dengan sqlite lewat library sqlite3 dan proses load data ke database menggunakan to_sql dari library pandas.



In [ ]:
try:
  con = sqlite3.connect('bank.db')
  cur = con.cursor()
  cur.execute(
    '''CREATE TABLE bank(
      "age"             INT,
      "job"             VARCHAR,
      "marital"         VARCHAR,
      "education"       VARCHAR,
      "default"         VARCHAR,
      "balance"         INT,
      "housing"         VARCAHR,
      "loan"            VARCHAR,
      "contact"         VARCHAR,
      "day"             INT,
      "month"           VARCHAR,
      "campaign"        INT,
      "pdays"           INT,
      "previous"        INT,
      "poutcome"        VARCHAR,
      "deposit"         VARCHAR
    );'''
    )
  con.commit()
  df.to_sql('BankMarket',con=con,if_exists='append',index=False)
except sqlite3.Error as err:
  print('Error while connecting to sqlite3', err)
finally:
  if con:
    con.close()
    print('The sqlite3 connection is closed')

The sqlite3 connection is closed
